#### 1. Import pandas library

In [19]:
import pandas as pd

#### 2. Import users table:

In [21]:
users = pd.read_csv('users_table.csv')
users.head(2)

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN


#### 3. Rename Id column to userId

In [22]:
users = users.rename(columns={'Id':'userId'})

#### 4. Import posts table:

In [23]:
posts = pd.read_csv('posts_table.csv')
posts.head(2)

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN


#### 5. Rename Id column to postId and OwnerUserId to userId

In [24]:
# When renaming more than one column you must add inplace parameter and set it to true
posts.rename(columns={'Id': 'postId', 'OwnerUserId': 'userId'}, inplace = True)
posts.head(1)

,postId,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,userId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [65]:
new_users = users.filter(['userId', 'Reputation', 'Views', 'UpVotes', 'DownVotes'], axis=1)
new_users.head()

new_posts = posts.filter(['postId', 'Score','userId','ViewCount','CommentCount'])
new_posts.head()

,postId,Score,userId,ViewCount,CommentCount
0,1,23,8.0,1278.0,1
1,2,22,24.0,8198.0,1
2,3,54,18.0,3613.0,4
3,4,13,23.0,5224.0,2
4,5,81,23.0,NaN,3


#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [127]:
merged = new_users.merge(new_posts)
merged.head(10)

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,NaN,0
1,-1,1,0,5007,1920,8576,0,NaN,0
2,-1,1,0,5007,1920,8578,0,NaN,0
3,-1,1,0,5007,1920,8981,0,NaN,0
4,-1,1,0,5007,1920,8982,0,NaN,0
5,-1,1,0,5007,1920,9857,0,NaN,0
6,-1,1,0,5007,1920,9858,0,NaN,0
7,-1,1,0,5007,1920,9860,0,NaN,0
8,-1,1,0,5007,1920,10130,0,NaN,0
9,-1,1,0,5007,1920,10131,0,NaN,0


#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [128]:
merged_null = merged.isnull().sum()
merged_null


userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [143]:
# I can clean the values from this column because there is already a column named Views for this purpose.
# If I fill in this column with zeros, I would have two columns with the same thing. 

drop_cols = list(merged_null[merged_null > 10000].index)     
drop_cols

# Axis = 1 (columns)  Axis = 0 (rows)

merged_dropped = merged.drop(drop_cols, axis = 1)


#get the same data set without the dropped columns

merged_dropped.head(10)


,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,CommentCount
0,-1,1,0,5007,1920,2175,0,0
1,-1,1,0,5007,1920,8576,0,0
2,-1,1,0,5007,1920,8578,0,0
3,-1,1,0,5007,1920,8981,0,0
4,-1,1,0,5007,1920,8982,0,0
5,-1,1,0,5007,1920,9857,0,0
6,-1,1,0,5007,1920,9858,0,0
7,-1,1,0,5007,1920,9860,0,0
8,-1,1,0,5007,1920,10130,0,0
9,-1,1,0,5007,1920,10131,0,0


#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [84]:
merged.dtypes

userId            int64
Reputation        int64
Views             int64
UpVotes           int64
DownVotes         int64
postId            int64
Score             int64
ViewCount       float64
CommentCount      int64
dtype: object

In [85]:
merged['ViewCount'] = merged['ViewCount'].astype('int64')  

In [86]:
merged.dtypes

userId          int64
Reputation      int64
Views           int64
UpVotes         int64
DownVotes       int64
postId          int64
Score           int64
ViewCount       int64
CommentCount    int64
dtype: object